In [1]:
#connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as img

import cv2
import itertools
import pathlib
import warnings
from PIL import Image
from random import randint
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix


from tensorflow import keras
from keras import layers
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

from distutils.dir_util import copy_tree, remove_tree

import os
print(os.listdir("/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02"))

print("TensorFlow Version:", tf.__version__)

['MODERATED DEMENTED', 'MILD DEMENTED', 'NON DEMENTED', 'VERY MILD DEMENTED']
TensorFlow Version: 2.12.0


In [6]:
!pip install efficientnet
from tensorflow import keras
from efficientnet.tfkeras import EfficientNetB7
model = EfficientNetB7(weights='imagenet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
268317848/268317848 [==============================] - 2s 0us/step


In [7]:
input_file = '/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02'
output_file = '/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02 SPLITTED '

In [8]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02 SPLITTED /train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=64
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02 SPLITTED /test",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/FINAL YEAR PROJECT/INPUT DATASET 02 SPLITTED /val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=64
)

Found 5119 files belonging to 4 classes.
Found 642 files belonging to 4 classes.
Found 639 files belonging to 4 classes.


In [9]:
class_names = train_ds.class_names
print(class_names)
train_ds

['MILD DEMENTED', 'MODERATED DEMENTED', 'NON DEMENTED', 'VERY MILD DEMENTED']


<_BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [10]:
import efficientnet.keras as efn

def build_efficientnet_b7_model():
    base_model = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    for layer in base_model.layers:
        layer.trainable = False

    model = keras.models.Sequential()
    model.add(keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
    model.add(base_model)

    # Add GlobalAveragePooling2D to reduce spatial dimensions
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(4, activation='softmax'))

    # Compile the model with the desired optimizer and loss function
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model


In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "Adam",metrics=["accuracy"])

In [12]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Create the EfficientNet B7 model
model = build_efficientnet_b7_model()

# Now, you can use the `model.fit()` function as before
hist = model.fit(train_ds, validation_data=val_ds, epochs=100, batch_size=64, verbose=1)

258434480/258434480 [==============================] - 5s 0us/step
Epoch 1/100
80/80 [==============================] - 900s 10s/step - loss: 1.0477 - accuracy: 0.5009 - val_loss: 0.9553 - val_accuracy: 0.5822
Epoch 2/100
80/80 [==============================] - 25s 300ms/step - loss: 0.9992 - accuracy: 0.5149 - val_loss: 0.9269 - val_accuracy: 0.5712
Epoch 3/100
80/80 [==============================] - 22s 273ms/step - loss: 0.9893 - accuracy: 0.5267 - val_loss: 0.9374 - val_accuracy: 0.5399
Epoch 4/100
80/80 [==============================] - 23s 279ms/step - loss: 0.9878 - accuracy: 0.5151 - val_loss: 0.8948 - val_accuracy: 0.5743
Epoch 5/100
80/80 [==============================] - 23s 280ms/step - loss: 0.9672 - accuracy: 0.5280 - val_loss: 0.9102 - val_accuracy: 0.5743
Epoch 6/100
80/80 [==============================] - 25s 305ms/step - loss: 0.9796 - accuracy: 0.5224 - val_loss: 0.9258 - val_accuracy: 0.5274
Epoch 7/100
80/80 [==============================] - 22s 274ms/step - 

In [ ]:
loss, accuracy = model.evaluate(test_ds)

In [13]:
hist = model.fit(train_ds,validation_data=val_ds,epochs=100, batch_size=64, verbose=1)

Epoch 1/100


ValueError: ignored

In [ ]:
# Plot accuracy and loss
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.plot([0, len(hist.history['accuracy'])-1], [test_acc, test_acc], '--', label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.plot([0, len(hist.history['loss'])-1], [test_acc, test_acc], '--', label='Test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()